In [1]:
import numpy as np
import pandas as pd
import gzip
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [2]:
import numpy as np
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df_orig = getDF('meta_Digital_Music.json.gz')

In [3]:
df_orig.head()

,asin,title,price,imUrl,related,salesRank,categories,description,brand
0,5555991584,Memory of Trees,9.49,http://ecx.images-amazon.com/images/I/51b5WDjd...,"{'also_bought': ['B000002LRT', 'B000002LRR', '...",{'Music': 939190},"[[CDs & Vinyl, New Age, Celtic New Age], [CDs ...",NaN,NaN
1,6308051551,Don't Drink His Blood,8.91,http://ecx.images-amazon.com/images/I/31LT2n7Q...,NaN,NaN,"[[Digital Music, Alternative Rock, Indie & Lo-...",NEW Combo BLUWAVS CD and FLAC FILE,NaN
2,7901622466,On Fire,11.33,http://ecx.images-amazon.com/images/I/21W29WZw...,"{'also_bought': ['B00000282A', 'B0084O8O9S', '...",{'Music': 58799},"[[CDs & Vinyl, Christian, Rock & Alternative],...",NaN,NaN
3,B0000000ZW,Changing Faces,23.64,http://ecx.images-amazon.com/images/I/51H3Bp-3...,"{'also_bought': ['B00000010Z', 'B0000039Q5', '...",{'Music': 68784},"[[CDs & Vinyl, Dance & Electronic], [CDs & Vin...",,
4,B00000016W,Pet Sounds,9.49,http://ecx.images-amazon.com/images/I/51pDGkXj...,"{'also_bought': ['B0025KVLTM', 'B00007FOMP', '...",{'Music': 77205},"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Po...",NaN,NaN


In [4]:
df = df_orig[["title", "description"]]

In [5]:
#drop rows with null descriptions
df = df.dropna(axis=0, subset=['description'])
#drop rows with empty descriptions
df = df[df['description'] != '']
#Drop rows with no title or nan title
df = df[df['title'] != '']
df = df[~df['title'].isna()]
#drop duplicate titles
df = df.drop_duplicates(subset=['title'])
#reset index
df = df.reset_index(drop=True)

In [6]:
df.head()

,title,description
0,Don't Drink His Blood,NEW Combo BLUWAVS CD and FLAC FILE
1,Harry,"A 1969 collection of songs about innocence, ex..."
2,Soulero,Soulero by Kenny BurrellThis product is manufa...
3,Ivory Pyramid,Ivory Pyramid by Ramsey LewisThis product is m...
4,That Secret Place,That Secret Place by Patti AustinThis product ...


In [7]:
tfidf = TfidfVectorizer(stop_words='english')

In [8]:
tfidf_matrix = tfidf.fit_transform(df['description'])

In [9]:
tfidf_matrix.shape

(4333, 32412)

In [10]:
from sklearn.metrics.pairwise import linear_kernel

In [11]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:
print(type(cosine_sim))
print(cosine_sim[1,1])

<class 'numpy.ndarray'>
0.9999999999999999


In [13]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates(inplace=False)

In [14]:
df.to_json("dataset.json")

In [15]:
#I made a custom class that can serialize our n-dimensional array
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [16]:
recommendation_data = {
  "matrix": cosine_sim,
  "indices": indices.to_dict()
}

In [17]:
with open('recommendation_data.json', 'w') as file:
    file.write(json.dumps(recommendation_data, cls=NumpyEncoder))

In [18]:
df.head()

,title,description
0,Don't Drink His Blood,NEW Combo BLUWAVS CD and FLAC FILE
1,Harry,"A 1969 collection of songs about innocence, ex..."
2,Soulero,Soulero by Kenny BurrellThis product is manufa...
3,Ivory Pyramid,Ivory Pyramid by Ramsey LewisThis product is m...
4,That Secret Place,That Secret Place by Patti AustinThis product ...


In [19]:
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[song_indices]

In [20]:
content_recommender("Funkytown")

1411                                          Antidote
3049                                         Funkytown
648                               Hearts &amp; Flowers
711                                       Looking East
1679                                      Garden Party
2950    Grand Theft Auto: Vice City, Vol. 4 - Flash FM
935                                          This Time
1364                                          Palomine
1875                                Acrobatic Tenement
1163               The Pointer Sisters - Greatest Hits
Name: title, dtype: object